In [1]:
# verified on 2/16/2025
# format of pass.txt
"""
google:your_email@gmail.com:your_google_app_password
yahoo:your_email@yahoo.com:your_yahoo_app_password
""";

In [2]:
import imaplib
import email
from email.header import decode_header
import pandas as pd
import re

# Yahoo IMAP settings
IMAP_SERVER = 'imap.mail.yahoo.com'
IMAP_PORT = 993
# EMAIL = 'your_email@yahoo.com'
# PASSWORD = 'your_app_password'

with open('pass.txt') as fin:
    strs=fin.readlines()
for s in strs:
    if s.split(":")[0]=='yahoo':
        EMAIL=s.split(":")[1].strip()
        PASSWORD=s.split(":")[-1].strip()
# EMAIL,PASSWORD    

In [3]:
# Connect to the server
mail = imaplib.IMAP4_SSL(IMAP_SERVER, IMAP_PORT)
mail.login(EMAIL, PASSWORD)

MAILBOX='INBOX'

status, _ = mail.select(MAILBOX)
if status != 'OK':
    print(f"Failed to select mailbox: {MAILBOX}")

In [4]:
start_date = '17-Feb-2025'
end_date = '18-Feb-2025'

search_criteria=f'(SINCE {start_date})'

status, messages = mail.uid('search',None, search_criteria)

# Search for emails within the date range
# status, messages = mail.uid('search',None, f'(SINCE {start_date} BEFORE {end_date})')
# status, messages = mail.search(None, f'SINCE "{start_date}"')
# status, messages = mail.search(None, 'ALL')

# sender_email = 'example@domain.com'
# # Search for emails from a particular address between dates
# search_criteria = f'(FROM "{sender_email}" SINCE "{start_date}" BEFORE "{end_date}")'
# status, messages = mail.search(None, search_criteria)

# Convert the message IDs to a list
target_uids = messages[0].split()

print(f"Found {len(target_uids)} emails.")


Found 32 emails.


In [5]:
results=[]
i=0
for uid in target_uids:
    i+=1
    # UID FETCH command
    status, data = mail.uid('fetch', uid, '(RFC822)')
    if status == 'OK' and data != [None]:
        r={}
        # Parse email content
        raw_email = data[0][1]
        msg = email.message_from_bytes(raw_email)
        sender=msg['From']
        print(f"\n{i} UID: {uid}")
        print(f"From: {sender}")
        
        # Decode headers
        subject, encoding = decode_header(msg['Subject'])[0]
        if isinstance(subject, bytes):
            try:
                subject = subject.decode(encoding if encoding else 'utf-8')
            except:
                subject="Decoding Error"        

        print(f"Subject: {subject}")
        
        r["id"]=uid
        r["sender"]=sender
        r["subject"]=subject
        results.append(r)           

    else:
        print(f"Email with UID {uid} not found")


1 UID: b'582448'
From: billing@sailinternet.com
Subject: Your Sail Internet invoice

2 UID: b'582453'
From: noreply@republicservices.com
Subject: Thank You for Your Payment

3 UID: b'582501'
From: Hongxia Chen <hongxiachen@hotmail.com>
Subject: Fwd: Re; 303 Anza St , Fremont 94539

4 UID: b'582511'
From: Hongxia Chen <hongxiachen@hotmail.com>
Subject: Re: Re; 303 Anza St , Fremont 94539

5 UID: b'582593'
From: "Henry Jones from Finance Media" <daily@your.financeinsidemedia.com>
Subject: America’s Greatest Threat—And How to Prepare

6 UID: b'582598'
From: "Facebook" <groupupdates@facebookmail.com>
Subject: Zhi Feng posted in 美國房屋裝修交流分享

7 UID: b'582603'
From: =?UTF-8?Q?Hidden_Wealth_=E2=80=A2_Arthur_White_=E2=80=A2_TEE?= <news@tee.theeconomicecho.com>
Subject: Tax Lien List 📃 • February 18

8 UID: b'582616'
From: "Facebook" <reminders@facebookmail.com>
Subject: You have 17 notifications about Jim and others

9 UID: b'582618'
From: "Amazon.com" <store-news@amazon.com>
Subject: Up to 40%

In [6]:
def get_email(str_sender):
    pattern = re.compile(r'<(.*?)>')
    match = re.search(pattern, str_sender)

    if match:
        # print("Found:", match.group())
        email=match.group(1)
    else:
        # print("Not found")
        email="NA"   

    sender, encoding = decode_header(str_sender)[0]
    if isinstance(sender, bytes):
        sender = sender.decode(encoding if encoding else 'utf-8')

    return email,sender   

In [7]:
df=pd.DataFrame(results)
df[["email","sender"]]=df.apply(lambda r:get_email(r['sender']),axis=1,result_type='expand')

In [8]:
df

,id,sender,subject,email
0,b'582448',billing@sailinternet.com,Your Sail Internet invoice,NA
1,b'582453',noreply@republicservices.com,Thank You for Your Payment,NA
2,b'582501',Hongxia Chen <hongxiachen@hotmail.com>,"Fwd: Re; 303 Anza St , Fremont 94539",hongxiachen@hotmail.com
3,b'582511',Hongxia Chen <hongxiachen@hotmail.com>,"Re: Re; 303 Anza St , Fremont 94539",hongxiachen@hotmail.com
4,b'582593',"""Henry Jones from Finance Media"" <daily@your.f...",America’s Greatest Threat—And How to Prepare,daily@your.financeinsidemedia.com
5,b'582598',"""Facebook"" <groupupdates@facebookmail.com>",Zhi Feng posted in 美國房屋裝修交流分享,groupupdates@facebookmail.com
6,b'582603',Hidden Wealth • Arthur White • TEE,Tax Lien List 📃 • February 18,news@tee.theeconomicecho.com
7,b'582616',"""Facebook"" <reminders@facebookmail.com>",You have 17 notifications about Jim and others,reminders@facebookmail.com
8,b'582618',"""Amazon.com"" <store-news@amazon.com>",Up to 40% off: Presidents' Day Sale ends tonight,store-news@amazon.com
9,b'582619',"""PayPal"" <PayPal@emails.paypal.com>","Yibing, your January account statement is avai...",PayPal@emails.paypal.com


In [9]:
list(df.email.unique())

['NA',
 'hongxiachen@hotmail.com',
 'daily@your.financeinsidemedia.com',
 'groupupdates@facebookmail.com',
 'news@tee.theeconomicecho.com',
 'reminders@facebookmail.com',
 'store-news@amazon.com',
 'PayPal@emails.paypal.com',
 'no-reply@accounts.google.com',
 'new@readysteadyprofit.com',
 'sail_internet@birdeye.com',
 'onlinebanking@ealerts.bankofamerica.com',
 'no-reply@e.siriusxm.com',
 'no-reply@em-s.dropbox.com',
 'USPSInformeddelivery@email.informeddelivery.usps.com',
 'editor@tddy.tradersdynasty.com',
 'news@t.keyincomejournal.com',
 'no-reply@email.alltrails.com',
 'news@wrrn.wealthyrepublicnews.com',
 'Costco@digital.costco.com',
 'mail@eg.expedia.com',
 'gain@wfp.wealthfuturepath.com',
 'Online-Do-Not-Reply@dmv.ca.gov',
 'growth@get.cashfortify.com',
 'cmueng-advancement@connect.cmu.edu',
 'thh@welcome.tradehavenhub.com',
 'team@asa.americanstockadvisors.com']

In [10]:
df.groupby(['email']).count().reset_index().sort_values('subject').tail(10)

,email,id,sender,subject
25,team@asa.americanstockadvisors.com,1,1,1
23,sail_internet@birdeye.com,1,1,1
19,no-reply@em-s.dropbox.com,1,1,1
20,no-reply@email.alltrails.com,1,1,1
22,reminders@facebookmail.com,1,1,1
24,store-news@amazon.com,1,1,1
2,Online-Do-Not-Reply@dmv.ca.gov,2,2,2
21,onlinebanking@ealerts.bankofamerica.com,2,2,2
11,hongxiachen@hotmail.com,2,2,2
1,NA,3,3,3


In [11]:
f2=['temu@updates.temuofficial.com', 'mylife.com',
   'americanstockadvisors',
   'thestockstandout','valoraeclipse','tradingfusionlab','metaversecapitalists','financialstoryarc',
   'bayareanewsgroup','recordstrivia','graynavigator',
   'investmentinnovate.com',
   'theinvestingtime.com',
   'a.richmarketwaves.com',
   'quantumcapitalvision',
   'invest','market','fortune','needtoknowfacts','fiscalforecastnet','milkymoneyway','quantumcapitalvision','fiscalforecastnet','financialstoryarc'
   'thestockstandout'
   'readysteadyprofit.com',
   'taxfoundation.com',
   'theretirementresearch.com',
   'customink.com',
   'oldamericanbroker.com',
   'smartsuccessstrategy.com',
   'theinvestingbox.com',
   'texasinnovationsmarket.com',
   'investingwithmind.com',
   'directionforexpo.com',
   'teamexpoexpert.com',
   'fiscalforecastnet',
   'r.moore@first.AmericanTruthCast.com', 
   'insight@the.paybackinvestigators.com',
   'edmond.p@daily.smoothtradingarea.com',
   'move@like.dealscatchers.com', 
   'news@tradelikegorillas.com',
   'ruffus.white@net.richpeoplenetworks.com',
   'raymondparks@tmp.titanmediapro.com', 
   'ae@e.ae.com',
   'daily@news.greenforesttrading.com',
   'daily@the.bankerbull.com', 
   'daily@blog.secretprofittools.com',
   'day@s.longliveamerican.com', 
#    'read@first.teamexpoexpert.com',
   'peter.s@the.directionforexpo.com', 
   'daily@inside.greenforesttrading.com',
   'l.barnett@wtf.wealthtimefuture.com',
   'email@digital.conservativeintel.com', 'gain@the.olympinvests.com',
   'hunter.wells@tip.profitabletradertips.com',
   'check@tips.sportinvestorsleague.com',
   'actual@Ii.investmentinnovate.com',
   'editor@nowcc.notesonwealthcreation.com',
   'editorial@wsi.wisetradeinvesting.com',
   'news@fn.fin-kit.com',
   'info@tfm.thefinancememories.com', 
   'news@daily.financiffy.com',
   'mailing@ceg.certaineconomicgrowth.com',
   'today@art.americansruletrading.com',
   'daily@mcp.moderncryptoparadise.com',
   'daily@d.tradetacticsnavigator.com',
#    'j.blume@mc.metaversecapitalists.com',
   'news@from.thegreatassets.com', 
   'ronaldvin@a.capitaliststoday.com']

In [12]:
f1=['walmartcustomerexperience@express10.medallia.com',
    #    'fandango@movies.fandango.com',
       'ftowdklvjeqagcszbymr@plugins.homemakingguide.com',
       'gain@the.olympinvests.com', 
    #    'noreplyhr@fb.com',
    #    'Costco@digital.costco.com', 
       'raymondparks@tmp.titanmediapro.com',
       'read@first.teamexpoexpert.com',
       'e-news@email.bayareanewsgroup.com',
    #    'billpay@billpay.bankofamerica.com', 
    #    'no-reply@rentcafe.com',
    #    'onlinebanking@ealerts.bankofamerica.com',
    #    'store+28212789326@m.shopifyemail.com',
    #    'hello@news.expressvpn.com', 
    #    'NA', 
    #    'palmia@legacypartners.com',
       'editorial@wsi.wisetradeinvesting.com',
    #    'groupupdates@facebookmail.com', 
       'me@jasongee.site',
       'silhruettrgrzphsextup@plugins.homemakingguide.com',
       'contact@masterclassjournal.com',
       'notifications@d.lavendercollab.com', 
       'humble@pobriar.com',
       'kent.harper@news.theinvestingbox.com',
       'cutleryandmore@cutleryandmore.com', 
       'contact@lectricebikes.com',
       'trivia@mail.wordstrivia.com',
       'just@info.globalfinancechanges.com',
       'global@news.takeprofitalerts.com',
       'email@the.deliciousinvesting.com', 
       'reply-to@e.digikey.com',
    #    'hongxiachen@hotmail.com', 
       'p.lynch@hmg.honeymoneygazette.com',
       'daily@the.bankerbull.com', 
       'news@s.stripesandstarsnews.com',
       'mylife@mail.mylife.com', 
       'marketing@emails.hertz.com',
       'move@like.dealscatchers.com',
    #    'j.blume@mc.metaversecapitalists.com',
       'from@global.investingcongress.com',
       'editor@star.starinvestingship.com',
       't.killian@sotmm.sharksofthemarket.com']

In [13]:

email_to_delete=f1+f2+['blog.secretprofittools.com','pro.milkymoneyway.com','ldii.longdistanceinvestings.com','first.AmericanTruthCast.com','today.investingwithmind.com','day.usastocksinsider.com',"best.marketexpopro.com",'net.richpeoplenetworks.com','fin.finacetechfuture.com','editor@nowcc.notesonwealthcreation.com','prisintol.com','tfm.thefinancememories.com','today.investingwithmind.com','ucc.urbancapitalist.com','digest.marketbroadview.com','tips.sportinvestorsleague.com','cottonbubblelay.com','deals@deals.vegas.com','find@via.healthyinvestingtips.com','webinar@newsletters.techonline.com','l.barnett@wtf.wealthtimefuture.com','get@real.multitradersmoves.com','mailing@ceg.certaineconomicgrowth.com','temu@updates.temuofficial.com','strategy@insights.FinancesDataDriven.com','ruffus.white@net.richpeoplenetworks.com','with@top.goldmineadvisors.com','your@very.smartsuccessstrategy.com','news@daily.investmentjourneys.com','check@tips.sportinvestorsleague.com','daily@digest.marketbroadview.com','q.hayes@choice.besttoinvestnews.com','news@daily.financiffy.com','daily@blog.secretprofittools.com','mail@your.usagoldmiracle.com','the.directionforexpo.com','daily@digest.marketbroadview.com']


In [14]:
subject_to_delete =['David Kant','FunWords Team','richard grant','stockpile saving','satelite earnings now','this token could','David Kant','Nextdish','hotels.com', 'Matthew Wehner','Elizabeth Harris', 'Trump','David Kant','Part-Time Work','Investors League','MileagePlus Partner','Russia in Shock','woot.com','Christopher Johnson','Tax-Free Cash Hack']

In [15]:
pattern1 = '|'.join(subject_to_delete)
pattern2 = '|'.join(email_to_delete)

In [16]:

dz=df[(df.sender.str.contains(pattern1, case=False)) | (df.email.str.contains(pattern2, case=False))]

In [17]:
dy=df[(~df.sender.str.contains(pattern1, case=False)) & (~df.email.str.contains(pattern2, case=False))]
dy.groupby(['email']).count().reset_index().sort_values('subject').tail(10)

,email,id,sender,subject
25,thh@welcome.tradehavenhub.com,1,1,1
22,reminders@facebookmail.com,1,1,1
23,sail_internet@birdeye.com,1,1,1
19,no-reply@em-s.dropbox.com,1,1,1
20,no-reply@email.alltrails.com,1,1,1
24,store-news@amazon.com,1,1,1
2,Online-Do-Not-Reply@dmv.ca.gov,2,2,2
11,hongxiachen@hotmail.com,2,2,2
21,onlinebanking@ealerts.bankofamerica.com,2,2,2
1,NA,3,3,3


In [18]:
dz.shape

(1, 4)

In [19]:
uids_to_delete=list(dz.id.unique())
uids_to_delete, len(uids_to_delete)

([b'582742'], 1)

In [20]:
# Delete emails
i=0
for uid in uids_to_delete:
    i+=1
    print(i,uid)
    try:
        status, response = mail.uid('store', uid, '+FLAGS', '\\Deleted')
        print(f"Delete status: {status}, Response: {response}")
    except:
        print(uid,"not working")
        
# Expunge deleted emails
mail.expunge()

1 b'582742'
Delete status: OK, Response: [b'10000 (FLAGS (\\Deleted \\Seen $NotJunk) UID 582742)']


('OK', [b'10000'])

In [21]:

# Close connection
mail.close()
mail.logout()

('BYE', [b'IMAP4rev1 Server logging out'])